# Parameters

In [22]:
dataset_metatada = "./networks/metadata.json"
opt = 'selected'
dvs = {
    "selected": {
        "TopologyZoo": ["Bellcanada", "Cogentco"],
        "DEFO": ["rf1239_real_hard"],
        "FatTree": ["FatTree8", "FatTree16", "FatTree32"]
    }
}
dataset_view = dvs[opt] if opt in dvs else None

# Parameters
detection_delay = [
    100000,  # nanoseconds = 100 microseconds
    1000000,  # nanoseconds = 1 ms
    10000000,  # nanoseconds = 10 ms
]
cache_lookup_delay = [
    0,  # nanoseconds
]
entry_installation_delay = [
    int(1e9/10000),  # nanoseconds = 100 microseconds
    int(1e9/1000),  # nanoseconds = 1 ms
    int(1e9/100),  # nanoseconds = 10 ms
]
packet_processing_delay = [
    10,  # nanoseconds
]
parameters = {
    'detection_delay': detection_delay,
    'cache_lookup_delay': cache_lookup_delay,
    'entry_installation_delay': entry_installation_delay,
    'packet_processing_delay': packet_processing_delay,
}

# Module Imports

In [23]:
import collections
import json
import math
import optparse
import os
import random
import time

import networkx as nx
import numpy as np
import pandas as pd

In [24]:
# MATPLOTLIB AND SEABORN
import matplotlib.pylab as pylab
from matplotlib.patches import Rectangle
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

%matplotlib inline

pylab.rcParams['font.size'] = 24
pylab.rcParams['figure.figsize'] = [3*x for x in [3.52, 2.64]]
pylab.rcParams['figure.dpi'] = 100
pylab.rcParams['errorbar.capsize'] = 3
pylab.rcParams['legend.fontsize'] = 12
pylab.rcParams['lines.linewidth'] = 2

gridcolor = '#bbbbbb'

zvalue = 2.576  # 99%

# Processing Functions

In [25]:
def proc_network(nm, cmd, dne, consts):
    # Build topology graph
    graph = nx.Graph()
    for node in range(nm['n_nodes']):
        graph.add_node(str(node))
    for edge in nm['edges']:
        graph.add_edge(str(edge['src']), str(edge['dst']), link_delay=edge['delay']*1000)
        graph.add_edge(str(edge['dst']), str(edge['src']), link_delay=edge['delay']*1000)
    comm_delay = dict(nx.all_pairs_dijkstra_path_length(graph, weight="link_delay"))
    comm_hops = dict(nx.all_pairs_dijkstra_path_length(graph))
    controller = min([(sum(v.values()), k) for k, v in comm_delay.items()])[1]
    time_to_compute = cmd["ttc_normal_forwarding"]*int(1e9)
    failure_sets = dne['failure_sets']
    n_entries = dne['n_entries']
    # Compute expected downtime
    res_cols = [
        "ssdn_lo", "ssdn_hi",
        "ssdn_detection_lo", "ssdn_detection_hi",
        "ssdn_notification_lo", "ssdn_notification_hi",
        "ssdn_computation_lo", "ssdn_computation_hi",
        "ssdn_transference_lo", "ssdn_transference_hi",
        "ssdn_installation_lo", "ssdn_installation_hi",
        "ssdn_update_lo", "ssdn_update_hi",
        
        "pcsdn_lo", "pcsdn_hi",
        "pcsdn_detection_lo", "pcsdn_detection_hi",
        "pcsdn_notification_lo", "pcsdn_notification_hi",
        "pcsdn_computation_lo", "pcsdn_computation_hi",
        "pcsdn_transference_lo", "pcsdn_transference_hi",
        "pcsdn_installation_lo", "pcsdn_installation_hi",
        "pcsdn_update_lo", "pcsdn_update_hi",

        "felix_lo", "felix_hi",
        "felix_detection_lo", "felix_detection_hi",
        "felix_coordination_lo", "felix_coordination_hi",
    ]
    df = pd.DataFrame(columns=res_cols)

    for fsid in failure_sets.keys():
        fs = failure_sets[fsid]
        ne = n_entries[fsid]
        total_ne = sum(ne.values())

        n_uniq_notifications = len(fs)
        nodes_local_to_failure = set()
        for u, v in fs:
            nodes_local_to_failure.add(str(u))
            nodes_local_to_failure.add(str(v))

        # Common SDN delays
        ## Notification Delay
        sdn_notification_delay_lo = min([comm_delay[u][controller] for u in nodes_local_to_failure])
        sdn_notification_delay_hi = max([comm_delay[u][controller] for u in nodes_local_to_failure])
        ## Transmission and Installation Delays
        sdn_update_delay_lo = 0
        sdn_update_delay_hi = 0
        sdn_transference_delay_lo = 0
        sdn_transference_delay_hi = 0
        sdn_installation_delay_lo = 0
        sdn_installation_delay_hi = 0
        for u in ne.keys():
            u_delay = comm_delay[u][controller] + ne[u]*consts['entry_installation_delay']
            # lo
            if u in nodes_local_to_failure:
                if u_delay > sdn_update_delay_lo:
                    sdn_update_delay_lo = u_delay
                    sdn_transference_delay_lo = comm_delay[u][controller]
                    sdn_installation_delay_lo = ne[u]*consts['entry_installation_delay']
            # hi
            if u_delay > sdn_update_delay_hi:
                sdn_update_delay_hi = u_delay
                sdn_transference_delay_hi = comm_delay[u][controller]
                sdn_installation_delay_hi = ne[u]*consts['entry_installation_delay']

        # Calculate Standard SDN approach downtime estimates
        ## Detection Delay
        ssdn_lo = consts["detection_delay"]
        ssdn_hi = consts["detection_delay"]
        ssdn_detection_lo = consts["detection_delay"]
        ssdn_detection_hi = consts["detection_delay"]
        ## Notification Delay
        ssdn_lo = ssdn_lo + sdn_notification_delay_lo
        ssdn_hi = ssdn_hi + sdn_notification_delay_hi
        ssdn_notification_lo = sdn_notification_delay_lo
        ssdn_notification_hi = sdn_notification_delay_hi
        ## Computation Delay
        ssdn_lo = ssdn_lo + time_to_compute
        ssdn_hi = ssdn_hi + n_uniq_notifications*time_to_compute
        ssdn_computation_lo = time_to_compute
        ssdn_computation_hi = n_uniq_notifications*time_to_compute
        ## Update Delay (Communication and Installation Delays)
        ssdn_lo = ssdn_lo + sdn_update_delay_lo
        ssdn_hi = ssdn_hi + sdn_update_delay_hi
        ssdn_transference_lo = sdn_transference_delay_lo
        ssdn_transference_hi = sdn_transference_delay_hi
        ssdn_installation_lo = sdn_installation_delay_lo
        ssdn_installation_hi = sdn_installation_delay_hi
        
        # Calculate Pre-Compute SDN approach downtime estimates
        ## Detection Delay
        pcsdn_lo = consts["detection_delay"]
        pcsdn_hi = consts["detection_delay"]
        pcsdn_detection_lo = consts["detection_delay"]
        pcsdn_detection_hi = consts["detection_delay"]
        ## Notification Delay
        pcsdn_lo = pcsdn_lo + sdn_notification_delay_lo
        pcsdn_hi = pcsdn_hi + sdn_notification_delay_hi
        pcsdn_notification_lo = sdn_notification_delay_lo
        pcsdn_notification_hi = sdn_notification_delay_hi
        ## Computation Delay
        pcsdn_lo = pcsdn_lo + consts["cache_lookup_delay"]
        pcsdn_hi = pcsdn_hi + n_uniq_notifications*consts["cache_lookup_delay"]
        pcsdn_computation_lo = consts["cache_lookup_delay"]
        pcsdn_computation_hi = n_uniq_notifications*consts["cache_lookup_delay"]
        ## Update Delay (Communication and Installation Delays)
        pcsdn_lo = pcsdn_lo + sdn_update_delay_lo
        pcsdn_hi = pcsdn_hi + sdn_update_delay_hi
        pcsdn_transference_lo = sdn_transference_delay_lo
        pcsdn_transference_hi = sdn_transference_delay_hi
        pcsdn_installation_lo = sdn_installation_delay_lo
        pcsdn_installation_hi = sdn_installation_delay_hi

        # Calculate Felix downtime estimates
        ## Detection Delay
        felix_lo = consts["detection_delay"]
        felix_hi = consts["detection_delay"]
        felix_detection_lo = consts["detection_delay"]
        felix_detection_hi = consts["detection_delay"]
        ## Coordination Delay
        felix_lo = felix_lo + consts['packet_processing_delay']  # considers nodes local to the failure
        felix_coordination_delay = 0
        for v in ne.keys():
            v_delay = None
            for u in nodes_local_to_failure:
                vu_delay = comm_hops[u][v]*consts['packet_processing_delay'] + comm_delay[u][v]
                if v_delay is None or vu_delay < v_delay:
                    v_delay = vu_delay
            if v_delay > felix_coordination_delay:
                felix_coordination_delay = v_delay
        felix_hi = felix_hi + felix_coordination_delay
        felix_coordination_lo = consts['packet_processing_delay']
        felix_coordination_hi = felix_coordination_delay
        
        # print(f"fsid={fsid} downtime={[ssdn_lo, ssdn_hi, pcsdn_lo, pcsdn_hi, felix_lo, felix_hi]}")
        df.loc[fsid] = [
            int(ssdn_lo), int(ssdn_hi),
            int(ssdn_detection_lo), int(ssdn_detection_hi),
            int(ssdn_notification_lo), int(ssdn_notification_hi),
            int(ssdn_computation_lo), int(ssdn_computation_hi),
            int(ssdn_transference_lo), int(ssdn_transference_hi),
            int(ssdn_installation_lo), int(ssdn_installation_hi),
            int(ssdn_transference_lo + ssdn_installation_lo), int(ssdn_transference_hi + ssdn_installation_hi),
        
            int(pcsdn_lo), int(pcsdn_hi),
            int(pcsdn_detection_lo), int(pcsdn_detection_hi),
            int(pcsdn_notification_lo), int(pcsdn_notification_hi),
            int(pcsdn_computation_lo), int(pcsdn_computation_hi),
            int(pcsdn_transference_lo), int(pcsdn_transference_hi),
            int(pcsdn_installation_lo), int(pcsdn_installation_hi),
            int(pcsdn_transference_lo + pcsdn_installation_lo), int(pcsdn_transference_hi + pcsdn_installation_hi),
            
            int(felix_lo), int(felix_hi),
            int(felix_detection_lo), int(felix_detection_hi),
            int(felix_coordination_lo), int(felix_coordination_hi),
        ]

    return df


In [26]:
def hstr(num):
    if num / 1e9 >= 1:
        return f"{num/1e9:.0f}s"
    if num / 1e6 >= 1:
        return f"{num/1e6:.0f}ms"
    if num / 1e3 >= 1:
        return f"{num/1e3:.0f}us"
    else:
        return f"{num:.0f}ns"

In [27]:
def main_loop(dm, params):
    base_dir = dm['base_directory']
    datasets = dm['datasets']

    total_n_cases = len(params["detection_delay"])*len(params["cache_lookup_delay"])*len(params["entry_installation_delay"])*len(params["packet_processing_delay"])
    cases_processed = 0
    print(f"({cases_processed}/{total_n_cases})")

    consts = {}
    for dd in params["detection_delay"]:
        consts["detection_delay"] = dd
        for ld in params["cache_lookup_delay"]:
            consts["cache_lookup_delay"] = ld
            for ed in params["entry_installation_delay"]:
                consts["entry_installation_delay"] = ed
                for ppd in params["packet_processing_delay"]:
                    consts["packet_processing_delay"] = ppd
                    consts_dir = f"./results/downtime/csvs/detection/{hstr(dd)}/cache_lookup/{hstr(ld)}/entry_installation/{hstr(ed)}/packet_processing/{hstr(ppd)}/"
                    os.makedirs(consts_dir, exist_ok=True)
            
                    # print(f"#######")
                    # print(f"Detection: {hstr(dd)}")
                    # print(f"Cache Lookup: {hstr(ld)}")
                    # print(f"Entry Installation: {hstr(ed)}")
                    # print(f"Packet Processing: {hstr(ppd)}")
                    # print()

                    for ds in datasets:
                        # print(f"Dataset={ds['name']}")
                        for nn in ds['networks']:
                            # print(f"  Network={nn}")
                            with open(f"{base_dir}/{ds['directory']}/{nn}.graph.json") as f:
                                nm = json.load(f)
                            with open(f"./results/routing/summary/{ds['name']}-{nn}.json") as f:
                                cmd = json.load(f)
                            with open(f"./results/routing/detailed-n-entries/{ds['name']}-{nn}.json") as f:
                                dne = json.load(f)
                            res = proc_network(nm, cmd, dne, consts)
                            # display(res)

                            # print(f"PC-SDN speedup: mean={res['pcsdn_speedup'].mean()} max={res['pcsdn_speedup'].max()}")
                            # print(f"Felix speedup over S-SDN: mean={res['felix_speedup_ssdn'].mean()} max={res['felix_speedup_ssdn'].max()}")
                            # print(f"Felix speedup over PC-SDN: mean={res['felix_speedup_pcsdn'].mean()} max={res['felix_speedup_pcsdn'].max()}")
                            
                            print(end=f"{nn} ")
                            res.to_csv(f"{consts_dir}/{ds['name']}-{nn}.csv")
                    print()
                    cases_processed = cases_processed + 1
                    print(f"({cases_processed}/{total_n_cases})")


# Compute Data

In [28]:
%%time

# Open full metadata
with open(dataset_metatada) as f:
    dm = json.load(f)

# Apply view
if dataset_view is not None:
    datasets = []
    for ds in dm['datasets']:
        if ds['name'] in dataset_view.keys():
            ds['networks'] = dataset_view[ds['name']]
            datasets.append(ds)
    dm['datasets'] = datasets

# Run main loop
main_loop(dm, parameters)

(0/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(1/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(2/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(3/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(4/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(5/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(6/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(7/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(8/9)
Bellcanada Cogentco rf1239_real_hard FatTree8 FatTree16 FatTree32 
(9/9)
CPU times: user 31min 16s, sys: 17 s, total: 31min 33s
Wall time: 31min 56s
